In [1]:
#load dataset
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from PIL import Image
import cv2
import zipfile
import numpy as np



In [2]:

import os
import numpy as np
import shutil
import random



# # Creating Train / Val folders 
dest_dir = 'data'
classes_dir = ['/AbdomenCT', '/BreastMRI', '/ChestCT', '/CXR','/Hand','/HeadCT']
root_dir= 'mnist'
test_ratio = 0.25

for cls in classes_dir:
    os.makedirs(dest_dir +'/train' + cls)
    os.makedirs(dest_dir +'/test' + cls)

    
    # Creating partitions of the data after shuffeling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)* (1-test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Testing: ', len(test_FileNames))

    #Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, dest_dir +'/train' + cls)

    for name in test_FileNames:
        shutil.copy(name, dest_dir +'/test' + cls)

Total images:  10000
Training:  7500
Testing:  2500
Total images:  8954
Training:  6715
Testing:  2239
Total images:  10000
Training:  7500
Testing:  2500
Total images:  10000
Training:  7500
Testing:  2500
Total images:  10000
Training:  7500
Testing:  2500
Total images:  10000
Training:  7500
Testing:  2500


In [3]:
### define train and validation directory

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "data/train"
training_datagen = ImageDataGenerator(rescale = 1./255)


VALIDATION_DIR = "data/test/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224,224),
    class_mode='categorical',
    batch_size=126
)

Found 44215 images belonging to 6 classes.
Found 14739 images belonging to 6 classes.


In [4]:
# load model and print vgg16 model summary
from tensorflow.keras.applications.vgg16 import VGG16
model=VGG16()
print(model)

2022-01-28 12:19:38.920022: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
print(model.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
# plot model summary
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='vgg.png')

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [7]:
from tensorflow.keras import layers

#we freez the other part of network
model=VGG16(include_top=False,input_shape=(224,224,3),weights=None)
for layer in model.layers:
    layer.trainable = False
 


In [8]:
#we add at the end network give 6 neurons

import tensorflow as tf 
# Flatten the output layer to 1 dimension
x = layers.Flatten()(model.output)

# Add a final sigmoid layer for classification
x = layers.Dense(6, activation='softmax')(x)

model = tf.keras.models.Model(model.input, x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
#### fit our model
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 5,
            epochs = 5,
            verbose = 1)

Epoch 1/5
5/5 [==============================] - ETA: 0s - loss: 1.7813 - accuracy: 0.1762

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
IMAGE_SIZE=[64, 64] 
training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(  train_FileNames , target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical',classes=['AbdomenCT', 'BreastMRI','ChestCT','CXR','Hand','HeadCT'])
test_generator = test_datagen.flow_from_directory( test_FileNames, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical',classes=['AbdomenCT', 'BreastMRI','ChestCT','CXR','Hand','HeadCT'])


classes = training_generator.class_indices
vgghist = model.fit(training_generator, validation_data = test_generator, steps_per_epoch = 10, epochs = 2)


In [ ]:
### plot train and validation set accuracy

import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)

plt.figure()


plt.show()